In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
#!git clone https://github.com/XuandongZhao/Unigram-Watermark
!pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 1.7 MB/s eta 0:00:00


In [ ]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [ ]:
cd zhao-Unigram-Watermark

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/zhao-Unigram-Watermark


In [ ]:
!pip install -q -r new_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.8 MB/s eta 0:00:00


In [ ]:
import argparse
from tqdm import tqdm
import json
import torch
import os
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LogitsProcessorList
from gptwm import GPTWatermarkLogitsWarper

In [ ]:
checkpoint = 'facebook/opt-1.3b'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
watermark_processor = LogitsProcessorList([GPTWatermarkLogitsWarper(fraction=0.5,
                                                                    strength=2.0,
                                                                    vocab_size=model.config.vocab_size,
                                                                    watermark_key=0)])

In [ ]:
cd '../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [ ]:
import string
p = set(string.printable)

In [ ]:
batch_size = 8
filename = 'zhao_no_attack.json'

df = pd.read_json('c4_selection.json', lines=True)

with open(filename, "r+" if os.path.exists(filename) else "w+") as f:
    with torch.no_grad():
        for i in tqdm(range(len(f.readlines()), len(df["input"]), batch_size)):
            line = df["input"][i:i+batch_size].values.tolist()
            tokenized_input = tokenizer(line, return_tensors='pt').to(model.device)

            output_tokens = model.generate(**tokenized_input,
                            logits_processor=watermark_processor,
                            max_new_tokens=250,
                            do_sample=True,
                            top_p=0.9
                            )

            # if decoder only model, then we need to isolate the
            # newly generated tokens as only those are watermarked, the input/prompt is not

            for j in range(len(output_tokens)):
                output_text = tokenizer.decode(output_tokens[j][50:], skip_special_tokens=True)

                _line = ''.join(filter(lambda x: x in p, line[j])).replace('\n', '\\n').replace('"', '\\"')
                _output_text = ''.join(filter(lambda x: x in p, output_text)).replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "continuation": "{_output_text}"}}\n')

100%|██████████| 64/64 [14:33<00:00, 13.64s/it]
